# Proposition d'algorithme de compression JPEG

### Léonie CROS, Alexandre WERLEN et Julie ZHAN

Nous allons présenter notre implémentation du processus de compression JPEG et les résultats expérimentaux. Nous avons choisi d'implémenter la compression JPEG en utilisant le paradigme de la programmation orientée objet en Python. Cette approche permet une structuration claire du code et facilite la manipulation des images en blocs distincts.

### 1) Préliminaires

Tout d'abord, il nous faut importer les différentes librairies dont nous aurons besoin :
- scipy.fftpack pour effectuer la DCT et l'inverse DCT
- PIL.Image pour ouvrir les images et récupérer les informations liées aux pixels de données
- numpy pour gérer les tableaux
- matplotlib.pyplot pour visualiser les résultats
- matplotlib.legend_handler.HandlerTuple pour gérer les légendes

In [6]:
#Importation des modules utilisés dans le code
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerTuple
from scipy import fftpack as ft
from PIL import Image as im

Puis c'est à vous de jouer ! Pour pouvoir faire tourner ce code sur votre propre ordinateur il vous faut télécharger les différents fichiers d'images tests présents dans le dossier partagé et initialiser correctement le répertoire parent. Assurez-vous que le répertoire de travail est bien celui où se trouve les images.

In [7]:
#Installation du parent directory
os.chdir("/Users/julie/Downloads/mémoireJPEG/Memoire_L3")

### 2) Définition de la Classe _Image couleur_

Dans le standard JPEG, l’image est souvent transformée d’un espace RGB (Rouge, Vert, Bleu) vers un espace où l’information est séparée en :
- Luminance (Y): représente la luminosité
- Chrominance (Cr et Cb): représente les informations de couleur (composantes bleue et rouge)
La classe Image couleur a donc pour rôle de charger une image en couleur et de la séparer en ces composantes avant la compression.

Pour la suite, on ne gardera que la luminance car l’œil humain est plus sensible aux variations de luminance qu’aux variations de couleur, et nous travaillons essentiellement sur des images en noir et blanc.

In [8]:
#Méta-classe image en couleur
class Image_couleur:
    def __init__(self,chemin):
        self.img=im.open(chemin) #image ouverte
        self.pix_val = list(self.img.getdata()) #liste des pixels sous forme (R,G,B)
        self.luminance=[]
        self.chrominance1=[]
        self.chrominance2=[]
        
    def division_en_3(self): #Sépare chaque pixel en luminance et en deux composantes de chrominance
        if type(self.pix_val[0])==tuple:
            for i in self.pix_val:
                self.luminance.append(np.mean(i))
                self.chrominance1.append(i[0]/2+i[1]/2)
                self.chrominance2.append(i[0]/2+i[2]/2)
            self.chrominance1=np.array(self.chrominance1)
            self.chrominance2=np.array(self.chrominance2)
            self.chrominance1=self.chrominance1.reshape(self.img.size)
            self.chrominance2=self.chrominance2.reshape(self.img.size)
        else:
            self.luminance=self.pix_val
        self.luminance=np.array(self.luminance)
        self.luminance=self.luminance.reshape(self.img.size[1],self.img.size[0]) 
        return self

### 3) Définition de la classe _Image_

La classe Image est responsable de la gestion de l'image entière et de sa division en blocs de $8\times8$. Chaque bloc sera traité séparément pour appliquer la compression JPEG.

Nous avons défini plusieurs attributs pour cette classe. A l'image à traiter est affecté le type Image, qui dispose de plusieurs attributs:
- contenu : contient le tableau numpy des valeurs de l'image (matrice de pixels)
- lenx : longueur des lignes de contenu
- leny : longueur des colonnes de contenu
- blocs : liste des blocs de 8 par 8 de type Bloc8x8 de contenu

Cette classe possède plusieurs méthodes. La plus importante est _division_ qui permet de diviser l'image à traiter en blocs de 8 par 8. Cette méthode peut aussi s'appliquer quand les dimensions de l'image à traiter ne sont pas divisibles par 8. Dans ce cas-là, les blocs incomplets sont complétés en répétant les dernières lignes et colonnes pour garantir une structure correcte.

In [14]:
#Méta-classe image 
class Image:
    def __init__(self, nom, matrice):
        self.nom = nom
        self.contenu = np.array(matrice) # matrice de l'image originale
        self.lenx = len(matrice[0]) #longueur des lignes
        self.leny = len(matrice) #longueur des colonnes
        self.blocs = [] # liste des blocs 8x8
        self.rle=[] #liste RLE concaténée
        self.huffman="" #texte compressé en Huffman 
        self.dico_huffman={}
        self.inverse_huffman="" #Inversion du codage Huffman
        self.inverse_rle=[] #séquence RLE après décompression
        self.resultat=[] #image reconstruite
        self.coeff=[] #coefficients DCT
        self.cpt=[]
        self.erreur=0 #erreur moyenne
        self.var=0 #variance de l'erreur

    def division(self):
        y=0
        cpt=1
        while y+8<=self.leny:
            x=0
            while x+8<=self.lenx:
                self.blocs.append(Bloc8x8(str(cpt),self.contenu[y:y+8,x:x+8]))
                cpt+=1
                x+=8
            if x!=self.lenx:
                L=[list(self.contenu[z,x:self.lenx])+[self.contenu[z,self.lenx-1] for i in range(8-(self.lenx-x))] for z in range(y,y+8)]
                self.blocs.append(Bloc8x8(str(cpt),np.array(L)))
                cpt+=1
            y+=8
        if y!=self.leny:
            x=0
            while x+8<=self.lenx:
                L=[list(self.contenu[z,x:x+8]) for z in range(y,self.leny)]+[list(self.contenu[self.leny-1,x:x+8]) for z in range(8-(self.leny-y))]
                self.blocs.append(Bloc8x8(str(cpt),np.array(L)))
                cpt+=1
                x+=8
            if x!=self.lenx:
                L=[]
                for z in range (y,self.leny):
                    L.append(list(self.contenu[z,x:self.lenx])+[self.contenu[z,self.lenx-1] for i in range(8-(self.lenx-x))])
                L=L+[L[-1] for z in range(8-(self.leny-y))]
                self.blocs.append(Bloc8x8(str(cpt),np.array(L)))         

    def compressI(self,Q):
        for b in self.blocs:
            b.compress(Q)
            self.rle.extend(b.rle)
        self.huffman,self.dico_huffman=huffman(self.rle)
        return self
        
    def recollage(self):
        M=0
        nx=self.lenx//8
        if self.lenx%8:
            nx+=1
        ny=self.leny//8
        if self.leny%8:
            ny+=1
        for y in range(ny):
            L=0
            for x in range(nx):
                if type(L)==int:
                    L=self.blocs[x+nx*y].comp
                else:
                    L=np.concatenate((L,self.blocs[x+nx*y].comp),axis=1)
            if type(M)==int:
                M=L
            else:
                M=np.concatenate((M,L),axis=0)
        self.resultat=M[:self.leny,:self.lenx]
        return self
        
    def metric(self,k,i,B): 
        self.erreur=np.mean(abs((self.resultat.astype(float)-self.contenu)))
        self.var=np.var((self.resultat.astype(float)-self.contenu))
        if B:
            Table[k][i].append((round(len(self.huffman)/(self.lenx*self.leny * 8),3),round(self.erreur,3),round(self.var,3)))
        else:
            print("Avant : {} bits / Après : {} bits,\n Taux de compression: {}\nErreur Moyenne: {} \n Variance de l'erreur: {}".format(self.lenx*self.leny * 8, len(self.huffman),round(len(self.huffman)/(self.lenx*self.leny * 8),2),round(self.erreur,3),round(self.var,3)))


    def coeffcount(self):
        for j in range(max(self.coeff)):
            self.cpt.append(self.coeff.count(j))
        return self.cpt


    def decompressI(self,Q,showimage=False):
        self.inverse_huffman=huffman_inverse(self.huffman,self.dico_huffman)
        for i in self.inverse_huffman:
            if type(i)==list:
                self.inverse_rle.extend([0]*i[1])
            else :
                self.inverse_rle.append(i)
        
        self.coeff=self.inverse_rle
        self.inverse_rle=np.array(self.inverse_rle)
        self.inverse_rle=self.inverse_rle.reshape((len(self.inverse_rle)//64,64))
        i=0
        for b in self.blocs:
            b.zigzaged=self.inverse_rle[i]
            b.inv_zigzag()
            i+=1
        for b in self.blocs:
            b.decompress(Q)
            self.erreur=self.erreur+b.erreur
        self.erreur=self.erreur/i
        self.recollage()
        if showimage:
            self.show_full_image()

    def show_full_image(self): #Représente l'image 
            new_im = im.fromarray(abs((self.resultat.astype(float)-self.contenu))*2)
            new_im.show()
            if new_im.mode != 'RGB':
                new_im = new_im.convert('RGB')
            new_im.save("Résultat/"+self.nom+"_erreur_residuel.jpg")
            new_im = im.fromarray(self.resultat.astype(float)) #Image compressée
            new_im.show()
            if new_im.mode != 'RGB':
                new_im = new_im.convert('RGB')
            new_im.save("Résultat/"+self.nom+"_img_residuel.jpg")
            new_im = im.fromarray(self.contenu.astype(float)) # Image originale
            new_im.show()
            return self


## 4) Définition de la classe _Bloc8x8_

La classe _Bloc8x8_ est utilisée pour représenter chaque bloc de $8\times8$ pixels et appliquer les différents étapes de la compresssion JPEG.

Chaque bloc contient plusieurs attributs, permettant de suivre les transformations successives de la compression.
Commençons tout d'abord par présenter quelques-uns de ses attributs :
- contenu : Contient le tableau numpy des valeurs du bloc
- comp : Image compilée, compressée après l'étape de DCT et de quantification
- DCT : Coefficients obtenus après la DCT
- line : Coefficients réorganisés selon l'ordre zigzag
- RLE : Image compressée avec le codage Run-Length Encoding
- erreur: Erreur de compression entre l'image originale et l'image compressée

Passons maintenant aux méthodes :
- Tout d'abord, la méthode _check\_matrice_ permet de vérifier si le bloc est bien une matrice de 8x8 et si les valeurs des pixels sont comprises entre 0 et 255.    
- La méthode _show_ est utilisée pour afficher l'image originale et l'image compressée afin de comparer la qualité de compression.
- Dans la méthode _calcul_, nous appliquons la Transformée en Cosinus Discrète bidimensionnelle puis la quantification à chaque bloc de 8x8. Pour cela, nous utilisons la fonction _dct_ du module _scipy.fftpack_ qui permet d'appliquer la DCT-II efficacement en mode orthonormé (_norm='ortho'_). La formule s'explique par le fait qu'une DCT-2D peut être décomposée en deux DCT-1D successives (appliquant sur les lignes et colonnes), et la normalisation permet de s'assurer que la transformation est orthonormée afin d'éviter les distorsions d'amplitude et d'avoir une transformation réversible correcte.
- Ensuite, la formule de la quantification est appliquée pour réduire les valeurs des hautes fréquences qui sont moins perceptibles. On arrondit les valeurs avec _round_ et les convertit en entiers en utilisant _astype_.
- Les coefficients DCT sont réorganisés dans la méthode _zigzag_.
- Dans la méthode _RLE_, on regroupe les séries des coefficients nuls pour compresser efficacement la représentation.
- _inverse\_calcul_ correspond au processus de décompression où on applique la formule de déquantification puis l'inverse DCT grâce à la fonction _idct_ issue du même module que _dct_. La déquantification multiplie chaque coefficient DCT par la matrice de quantification Q.
- Dans _score_, on évalue l'erreur moyenne absolue(MAE) entre le bloc compressé(_comp_) et original(_contenu_). L'erreur moyenne absolue représente la moyenne des différences absolues entre les valeurs de pixels d'origine et les valeurs des pixels compressés. Elle est définie par:
<h3><center>$MAE = \frac{1}{64}\sum_{i = 0}^{7}\sum_{j = 0}^{7} | \text{comp}[i,j]-\text{contenu}[i, j]|$</h3>
où dans un bloc $8\times8$, nous avons $contenu[i,j]$ la valeur d'un pixel dans le bloc original et $comp[i,j]$ la valeur d'un pixel dans le bloc compressé après l'étape de décompression.

In [10]:
class Bloc8x8: #Classe sur un bloc 8x8 de l'image
    def __init__(self, nom, matrice):  
        self.nom = nom 
        self.contenu = matrice #bloc 8x8 avec lequel on travaille type np.array
        self.valide=True
        self.comp = [] #image comprimé
        self.DCT = np.array([]) #coefficients après DCT 
        self.line = [] #coefficients réorganisés en zigzag
        self.rle=[] #encodage Run-Length
        self.zigzaged=[] 
        self.dezigzaged=[]
        self.bloc_reconstruit = [] #bloc reconstruit après IDCT
        self.erreur=0 #erreur de reconstruction

        
    def test(self):
        print("Nom: ", self.nom, "type: ", type(self.nom))
        print("Contenu: ", self.contenu, "type: ", type(self.contenu))
        print("Valide: ", self.valide) 
        print("Compilation: ", self.comp, "type: ", type(self.comp))
        print("DCT: ", self.DCT, "type: ", type(self.DCT))
        print("Line: ", self.line, "type: ", type(self.line))
        print("RLE: ", self.rle, "type: ", type(self.rle))
    
    
    def check_matrice(self): #Vérifie que la matrice est bien un bloc 8x8 avec des pixels entre 0 et 255
        if self.contenu.shape != (8,8):
            self.valide = False
            print("error")
        for i in range(0,8):
             for j in range(0,8):
                if self.contenu[i,j]> 255 or self.contenu[i,j]<0:
                    self.valide = False
                    print("error")
                    
                    
    def show(self, afficher_compression=True): #Affiche l'image originale et éventuellement le bloc compressé.
        self.check_matrice()
        if not self.valide:
            print("Bloc invalide.")
            return self   
        if afficher_compression:
            plt.figure(figsize=(8, 8))
            # Image originale
            plt.subplot(1, 2, 1)
            plt.imshow(self.contenu, cmap="gray", vmin=0, vmax=255)
            plt.colorbar(label="Intensité (0-255)", fraction=0.046)
            plt.title("Bloc original " + self.nom)
            plt.axis("off")         
            # Image reconstruite
            plt.subplot(1, 2, 2)
            plt.imshow(self.bloc_reconstruit, cmap="gray", vmin=0, vmax=255)
            plt.colorbar(label="Intensité (0-255)", fraction=0.046)
            plt.title("Bloc compressé " + self.nom)
            plt.axis("off")

            plt.tight_layout()
            plt.show()
        return self
  
            
    def calcul(self,Q): #Calcule la DCT 2D et applique la quantification avec la matrice Q
        self.DCT = ft.dct(ft.dct(self.contenu.T, norm='ortho').T,norm='ortho')
        self.DCT = np.round(self.DCT/Q).astype(int)
        return self.DCT
  
    def zigzag(self): #Réorganise les coefficients DCT selon l'ordre zigzag
        # M np.array de de dimension d*d
        sgn=1 #sens de parcours de la diagonale
        back=False #mot-clef début de la seconde moitié de la matrice
        for sum in range(2*8 -1):
            #itération sur la somme de i abscisse et j ordonnée de chaque coefficient de M ie les diagonales de M
            if sum==8:
                back=True
            #détermination de la liste des abscisses des coefficients de la diagonale à parcourir
            if sgn==-1:
                if not back:
                    R=range(sum+1)
                else:
                    R=range(sum-8+1,8)
            else:
                if not back:
                    R=range(sum,-1,-1)
                else:
                    R=range(8-1,sum-8,-1)
            for i in R:
                self.line.append(self.DCT[i,sum-i]) #i+j=sum donc j=sum-i
            sgn=-sgn
        return self.line
    
    
    def RLE(self): #Applique le Run-Length Encoding sur les coefficients zigzag
        k=-1 #marqueur initial k
        cpt=0
        for i in self.line: 
            if i==0:
                # cas initial (cas premier 0) pour i = 0: on crée la liste des 0
                if k!=0:
                    self.rle.append([0,1])
                    cpt+=1
                # 0 suivants
                else:
                    self.rle[cpt-1]=[0,self.rle[cpt-1][1]+1]
            # cas différent de 0
            else:
                self.rle.append(i)
                cpt+=1
            k=i
        return self
    
    def inv_zigzag(self): #Reconstruit la matrice depuis les coefficients zigzag
        sgn=-1
        L=[[] for i in range(8)]
        back=False
        cpt=0
        for i in range(1,2*8):
            if i==9:
                back=True
            if not back:
                D={j:self.zigzaged[cpt+j] for j in range(i)}
                n=len(D)
                if sgn==-1:
                    for j in range(n):
                        L[j].append(D[n-j-1])
                else:
                    for j in range(i):
                        L[j].append(D[j])
            else:
                D={j:self.zigzaged[cpt+j] for j in range(8-i%8)}
                n=len(D)
                if sgn==1:
                    for j in range(n):
                        L[7-j].append(D[n-j-1])
                else:
                    for j in range(n):
                        L[7-j].append(D[j])
            cpt+=n
            sgn=-sgn
        self.dezigzaged=np.array(L)
        return self
    
    
    def inverse_calcul(self,Q): #Déquantifie et applique l'inverse de la DCT pour reconstruire le bloc
        self.dequantifier = self.DCT*Q
        self.comp = ft.idct(ft.idct(self.dequantifier.T, norm='ortho').T,norm='ortho')
        return self.comp
  

    def score(self): #Calcule l'erreur moyenne absolue sur le bloc
        self.erreur=(np.sum(abs(self.comp-self.contenu))/64)
        return self
    
    def compress(self,Q): #Effectue les étapes de compression
        self.calcul(Q)
        self.zigzag()
        self.RLE()
        return self

    def decompress(self,Q): #Effectue les étapes de décompression
        self.inverse_calcul(Q)
        self.score()
        return self

### 5) Codage Huffman

La cellule suivante contient un algorithme de codage Huffman adapté de _https://gist.github.com/GautierLePire/ee104b066f184f9c62bc89e87e494c06_ par Gaultier Lepire.

In [11]:
def compterOccurences(liste):
    """
    Renvoie une liste qui associe à chaque caractère son nombre d'apparitions.
    Chaque lettre est dotée d'un poids (son nombre d'occurences). 
    Plus son poids est élevé, plus elle sera légère en mémoire.
    L'objectif est d'échanger de la puissance de calcul contre de l'espace de stockage pour bien compresser.
    """
    lettres={}
    for i in liste:
        j=i
        if type(i)==list:
            j="$"+str(i[1])
        if j in lettres :
            lettres[j]+=1
        else:
            lettres[j]=1
    alphabet=[]
    for i in lettres:
        alphabet.append([lettres[i],i])
    return alphabet

def creerArbre(alphabet):
    """
    Crée un arbre binaire à partir des lettres et de leur poids.
    On choisit de représenter un arbre de la façon suivante :
      * Une feuille est un 2-uplet : le nombre d'occurences et la lettre.
        On notera que compterOccurences renvoie une liste de feuilles.
      * Un noeud est un 3-uplet : la somme des occurences de toutes
        les feuilles descendantes, le fils gauche et le fils droit.
    Ensuite, on construit l'arbre en piochant les deux noeuds de poids
    le plus faible, on en fait un nouveau noeud que l'on remet dans le tas.
    On s'arrête dès qu'il reste un unique noeud (qui est l'arbre voulu).
    """
    # On commence par enlever les lettres qui ne sont pas présentes
    noeuds = alphabet
    # Puis on récupère les deux noeuds (ou feuilles) de poids le plus faible,
    # et on en fait un noeud, de poids la somme des deux petits poids
    # On boucle tant qu'il y a reste au moins deux noeuds
    l = len(noeuds)
    while l >= 2:
        # Indice et noeud des minima des poids
        # On initialise avec les deux premières valeurs
        petitMin = (0, noeuds[0])
        grandMin = (1, noeuds[1])
        for i in range(2, l):
            if noeuds[i][0] <= petitMin[1][0]:  # poids < petitMin < grandMin
                grandMin = petitMin
                petitMin = (i, noeuds[i])
            elif noeuds[i][0] <= grandMin[1][0]:  # petitMin < poids < grandMin
                grandMin = (i, noeuds[i])
        nouveauNoeud = (
            petitMin[1][0] + grandMin[1][0],
            noeuds[petitMin[0]],
            noeuds[grandMin[0]]
        )
        # On enlève les deux noeuds (ou feuilles) précedentes et on ajoute le nouveau noeud
        noeuds[petitMin[0]] = nouveauNoeud
        noeuds.pop(grandMin[0])
        # On a au final un noeud de moins (-2 +1)
        l -= 1
    # À cet instant il ne reste plus qu'un noeud, qui est la racine de l'arbre de Huffman
    return noeuds[0]

"""
On remarquera que moins une lettre est présente, plus elle est profonde dans l'arbre.
L'idée maintenant va être d'attribuer à chaque lettre un code en binaire de la façon
suivante :
  * la longueur du code est égale à la profondeur dans l'arbre
  * pour chaque lettre (donc chaque feuille), on part de la racine et on ajoute un 0
    si on prend la branche de gauche, un 1 si on passe par celle de droite.
Ainsi, plus une lettre est fréquente, plus son code binaire est court.
"""

def creerDico(arbre):
    """
    Renvoie un dictionnaire {lettre: code binaire}.
    On va explorer l'arbre à l'aide d'une file : si on rencontre une feuille,
    on la traite, si on rencontre un noeud, on ajoute les deux branches à la file.
    Le premier composant d'un élément de la file est le code binaire jusqu'à cet élément,
    le second est un noeud ou une feuille.
    """
    fileExploration = [("", arbre)]
    dico = {}
    l = 1
    # On boucle tant que la file n'est pas vide
    while l >= 1:
        code, truc = fileExploration.pop(0)  # On défile le premier élément
        l -= 1
        if len(truc) == 2:  # C'est une feuille
            dico[truc[1]] = code  # On ajoute la lettre et son code au dico
        elif len(truc) == 3:  # C'est un noeud
            # On continue l'exploration en respectant la règle pour obtenir le code :
            # Gauche -> 0, droite -> 1
            fileExploration.append((code + "0", truc[1]))
            fileExploration.append((code + "1", truc[2]))
            l += 2

    return dico

def huffman(liste):
    """
    On se contente de remplacer les lettres du texte par le code binaire
    obtenu à l'aide de la fonction creerDico.
    """
    alphabet = compterOccurences(liste)
    arbre = creerArbre(alphabet)
    dico = creerDico(arbre)
    texteCompresse = ""
    for i in liste:
        if type(i)==list:
            texteCompresse += dico["$"+str(i[1])]
        else:
            texteCompresse += dico[i]
    # On n'oublie pas de renvoyer aussi le dictionnaire, sinon il sera impossible de décompresser le texte
    return texteCompresse, dico

def huffman_inverse(texteCompresse, dicoRetourne):
    """
    Décompresse un texte à l'aide de son dico.
    Une fois encore, on utilise une file. C'est un outil très puissant
    qui permet de ne jamais écrire de fonction récursive. Chaque élément
    de la file est un 2-uplet, le premier élément est le texte décompressé
    jusque là, le second est le code binaire restant à décompresser.
    """
    # On retourne le dico
    dico = {v: k for (k, v) in dicoRetourne.items()}
    # Nombre maximum de bits d'un caractère compressé
    limite = max(len(k) for k in dico.keys())
    fileExploration = [("", texteCompresse)]
    l = 1
    a=[]
    while l >= 1:
        fait, restant = fileExploration.pop(0)  # On défile le premier élément
        l -= 1
        # On regarde si la décompression est terminée
        if restant == "":
            return a
        # Sinon, on tente de remplacer les i premiers bits de restant par un caractère
        i = 0
        bits = ""
        for bit in restant:
            bits += bit
            i += 1
            if i > limite:
                # C'est pas la peine de continuer, bits est trop long
                # pour correspondre à un caractère
                break
            elif bits in dico:
                # On a la possibilité de remplacer quelques 0 et 1 par un caractère
                # alors on le fait, sans pour autant considérer que l'on a choisi
                # le bon remplacement
                if type(dico[bits])==str:
                    a.append([0,int(dico[bits][1:])])
                else:
                    a.append(dico[bits])
                fileExploration.append((fait+str(dico[bits]), restant[i:]))
                l += 1
                
                # Puis on continue à explorer les possibilités
    # Aucune décompression n'a fonctionné, on ne renvoie rien
    return None



### 6) Quantification

Voici ensuite différentes matrices de quantifications issues de _https://hal.science/hal-03858141/file/qtable.pdf_, permettant de réaliser des niveaux de compression différents. La Matrice _Q1_ correspond à une quantification minimale, la matrice _Q2_ à une quantification intermédiaire et la matrice _Q3_ à une quantification importatnte.

In [12]:
#Quantification minimale
Q1=[1,1,1,1,1,2,2,2,1,1,1,1,1,2,2,2,1,1,1,1,2,2,3,2,1,1,1,1,2,3,3,2,1,1,1,2,3,4,4,3,1,1,2,3,3,4,5,4,2,3,3,3,4,5,5,4,3,4,4,4,4,4,4,4]
#Quantification intermédiaire
Q2=[1,2,2,3,5,8,10,12,2,2,3,4,5,12,12,11,3,3,3,5,8,11,14,11,3,3,4,6,10,17,16,12,4,4,7,11,14,22,21,15,5,7,11,13,16,21,23,18,10,13,16,17,21,24,24,20,14,18,19,20,22,20,21,20]
#Quantification importante
Q3=[1,9,8,13,19,32,41,49,10,10,11,15,21,46,48,255,11,10,13,19,32,46,55,45,11,14,18,23,41,255,255,255,14,18,30,45,54,255,255,255,19,28,44,51,255,255,255,255,39,51,62,255,255,255,255,255,58,255,255,255,255,255,255,255]

Q1=np.array(Q1)
Q1=Q1.reshape((8,8))
Q2=np.array(Q2)
Q2=Q2.reshape((8,8))
Q3=np.array(Q3)
Q3=Q3.reshape((8,8))
def quantification(l,Q):
    return np.round(l*Q)


Maintenant, voici la dernière étape: l'affichage des résultats.
La fonction Affichage_resultats fait tourner le processus de compression sur l'image test et affiche: l'image originale, l'image compressée, ainsi que l'erreur moyenne en pixel.
Ici, la fonction s'exécute sur l'image numéro 6 du dossier _image_test_, mais si vous voulez tester sur d'autres images, il vous suffit simplement de changer le paramètre _n_ par le numéro correspondant à l'image que vous souhaitez compresser dans le dossier _image_test_.

In [18]:
#Extraction des données

Table=[]
def Affichage_resultats(n,T):
    title=["Black&White","Arbre","Montagne","Mère","Gravure","Libellulle","Kayak","Lunch","Eiffel","Maison","Poule","Poème","Fleur"]
    path="image_test/"+str(n)+".jpg"
    image_test=Image_couleur(path).division_en_3().luminance #extraire la luminance de l'image test
    Test=Image(title[n],image_test)
    Test.division()
    Test.compressI(Q2)
    Test.decompressI(Q2,T)
    return Test

# Exemple d'exécution sur l'image numéro 6 (Kayak)
Affichage_resultats(6,True).metric(1,1,False)

Avant : 3027456 bits / Après : 883346 bits,
 Taux de compression: 0.29
Erreur Moyenne: 0.276 
 Variance de l'erreur: 0.165


Si vous testez plusieurs images différentes vous vous rendez vite conpte que la qualité de la compression varie d'une image à l'autre. En effet, le panel d'images à votre disposition à été sélctionné de façon à être représentatif des diverses situations possibles dont des cas limites dans lesquels le processus JPEG est moins approprié. Commençons par un cas particulièrement favorable: l'image du kayakiste. Cette image est constituée de grandes zones de couleur homogènes et de transitions douces entre les différents niveaux de luminosité. A l’oeil nu, aucune différence notable ne peut être observée entre l’image originale et l’image compressée.
Au contraire, le texte est un cas plus litigieux. En effet, la séparation brutale très fine entre les pixels blancs et les pixels noirs est une situation particulièrement inadaptée à la compression par DCT. Après application de l’algorithme, le texte est plus flou et moins lisible à distance.